In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option('display.colheader_justify', 'center')
pd.options.display.max_colwidth = 100

In [2]:
from libForClustering.data_structures import *
from libForClustering.preprocessing import *
from libForClustering.feature_processing import *

# utils

In [3]:
def printFormatdfCxtFToRCFT(df, title):
    print("FormalContext " + title)
    print("||")
    for index, row in df.iterrows():
        print("|" + str(row[0]) + "|")
    print()


def printFormatdfCxtRToRCFT(df, title, source, target):
    print("RelationalContext " + title)
    columns = list(df.columns.values)
    print("source " + source)
    print("target " + target)
    print("scaling exist")
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    print(strCol)
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        print(strtmp)
    print()


def retFormatdfCxtFToRCFT(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    strRet += ("||") + "\n"
    for index, row in df.iterrows():
        strRet += ("|" + str(row[0]) + "|") + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtRToRCFT(df, title, source, target):
    strRet = ""
    strRet += ("RelationalContext " + title) + "\n"
    columns = list(df.columns.values)
    strRet += ("source " + source) + "\n"
    strRet += ("target " + target) + "\n"
    strRet += ("scaling exist") + "\n"
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet


def retFormatdfCxtMatrixed(df, title):
    strRet = ""
    strRet += ("FormalContext " + title) + "\n"
    columns = list(df.columns.values)
    strCol = "||"
    for column in columns:
        strCol += str(column) + "|"
    strRet += (strCol) + "\n"
    for index, row in df.iterrows():
        strtmp = "|" + str(index) + "|"
        for cell in row:
            strtmp += str(cell) + "|"
        strRet += (strtmp) + "\n"
    strRet += "\n"
    return strRet


def union(lst1, lst2):
    final_list = list(set(lst1) | set(lst2))
    return final_list


# data is a dataframe of array
def ctxRelPerCtxForms(df_ctxFormX, data):

    tabRet = []
    # On parcours toutes les lignes des données
    for indexData, rowData in data.iterrows():
        tabContains = []
        # On parcours toutes les lignes du contexte formel
        for indexCtx, rowCtx in df_ctxFormX.iterrows():
            # On compare si l'élément est présent
            if rowCtx[0] in rowData[0]:
                tabContains.append("x")
            else:
                tabContains.append("")
        tabRet.append(tabContains)
    return tabRet

## Produits

In [4]:
def uniformRoles(roles):
    # Uniformisation des roles
    roles = list(map(lambda x: x.replace('admin', 'administrateur'), roles))
    roles = list(
        map(lambda x: x.replace('administrateuristrateur', 'administrateur'),
            roles))
    roles = list(map(lambda x: x.replace('Presto', ''), roles))
    roles = list(map(lambda x: x.replace('PinkLady', ''), roles))
    roles = list(map(lambda x: x.replace('producteurdepomme', 'producteur'),
                     roles))
    roles = list(map(lambda x: x.replace('utilisateurde', 'utilisateur'), roles))
    roles = list(map(lambda x: x.replace('testeurde', 'testeur'), roles))
    roles = list(
        map(lambda x: x.replace('utilisateurVigneouAmandes', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('utilisateurAmandes', 'utilisateur'), roles))
    roles = list(
        map(lambda x: x.replace('utilisateurVigneouAmandes', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('utilisateurCropwin//Banana', 'utilisateur'),
            roles))
    roles = list(
        map(lambda x: x.replace('responsableCercoban', 'responsable'), roles))
    roles = list(
        map(
            lambda x: x.replace(
                'administrateurjepeuxaffecter(désaffecter)unesaisonàunutilisateur.',
                'administrateur'), roles))
    roles = list(
        map(
            lambda x: x.replace(
                "administrateurjepeuxgérerlessaisonsdel'application(CRUD)",
                'administrateur'), roles))
    rolesUndupli = roles
    # Simplification des roles
    for role in roles:
        if ("administrateur" in role):
            rolesUndupli = list(
                map(lambda x: x.replace(role, 'administrateur'), roles))
        if ("utilisateur" in role):
            rolesUndupli = list(
                map(lambda x: x.replace(role, 'utilisateur'), roles))
        if ("technicien" in role):
            rolesUndupli = list(map(lambda x: x.replace(role, 'technicien'),
                                    roles))
    return rolesUndupli

# Clusturing

In [5]:
def remplacingRole(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace("As ", "")
    sentence = sentence.replace("as ", "")
    sentence = sentence.replace("a ", "")
    sentence = sentence.replace("an ", "")
    sentence = sentence.replace("( ", "")
    sentence = sentence.replace(") ", "")
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace(" admin", "administrator")
    sentence = sentence.replace(" ", "")
    return sentence

def remplacingFeature(sentence):
    sentence = sentence.replace("'", "")
    sentence = sentence.replace(",", "")
    sentence = sentence.replace("(", " ")
    sentence = sentence.replace(")", " ")
    sentence = sentence.replace("  ", " ")
    return sentence

In [6]:
f = open("data/datasetEN.txt", "r", encoding="UTF-8")
data = f.read()
f.close()
documents = Documents()
data = data.split('\n')

In [7]:
data

['As a farmer, I can refresh the predicted weather',
 'As a farmer, I can CRUD plots',
 'As a farmer, I can edit the parameters of a plot (current season)',
 'As a farmer, I can sort my plots in the list',
 'As a farmer, I can filter my plots in the list',
 'As a farmer, I can export observation data for a plot',
 'As a farmer, I know when my plots will be in danger',
 'As a farmer, I can manage my irrigations and recommendations in my favorite unit',
 'As a farmer, I can choose my preferred irrigation unit in my user-settings',
 'As a farmer, I can view my irrigation recommendations in my favorite unit',
 'As a admin, I can CRUD a farmer',
 'As a admin, I can relaunch all failed simulation',
 '']

In [8]:
roleData = []
featureData = []
for us in data:
    usSplit = us.split("I")
    roleData.append(remplacingRole(usSplit[0]))
    featureData.append(remplacingFeature(("".join(usSplit[1:]))[1:]))
del roleData[-1]
roleData = uniformRoles(roleData)
del featureData[-1]
dfRoleFeatureSuperF = pd.DataFrame(data={"Role": roleData})

In [9]:
try:
    documents = Documents()
    data = featureData
    try:
        while(True):
            data.remove('')
    except ValueError:
        pass
    for d in data:
        documents.add_document(Document(d))
except:
    print("ERROR - ReadFromFileError ("+filename+")")

In [10]:
dataset = Dataset()
preprocessing = Preprocessing()
tagged_documents = preprocessing.prepare_parts_of_user_stories(
    documents, False, False, True, False, True, True, "en")

Loading model...
Processing synsets...
[====================]


In [11]:
featureDataList = []
for doc in tagged_documents:
    featureDataList.append(" ".join(doc[0]))
dfRoleFeatureSuperF["Feature"] = featureDataList

In [12]:
featureProcessing = FeatureProcessing()
# dict = map id mot
# doc_term_matrices = idMot par document
dictionary, doc_term_matrices = featureProcessing.generate_dict_dtm(documents)

In [13]:
# get initial features
#initial_features = featureProcessing.initial_features(documents)
#print(type(initial_features[0]))

initial_features = []
for doc in documents:
    initial_features.append(Feature(doc))

In [14]:
# tranform into following lists (feature names, feature keywords, feature tags)
feature_list = []
for feature, tokens, tags, dtm in zip(initial_features,
                                      documents.tokenized_documents,
                                      tagged_documents, doc_term_matrices):
    feature_list.append((feature, tokens, tags, dtm))
grouped_lists = (initial_features, documents.tokenized_documents,
                 tagged_documents, doc_term_matrices)

# grouped_lists = initial_features, documents.tokenized_documents, tagged_documents, doc_term_matrices

In [15]:
vectors = featureProcessing.get_vectors_doc2vec(grouped_lists[2],
                                                grouped_lists[1])

c = Cluster()
num_clusters = c.get_optimal_elbow_num(vectors, 2, len(vectors), 1)
num_clusters

6

In [16]:
num_repeats = 200
max_iter = 2000
clusterer = Cluster()
vectors = featureProcessing.get_vectors_doc2vec(grouped_lists[2],
                                                grouped_lists[1])
clusterer.skl_kmeans(num_clusters, num_repeats, max_iter)
clusters = clusterer.skl_kmeans_fit_and_cluster_vectors(vectors)
sorted_clusters = clusterer._sort_features(clusters, feature_list,
                                           num_clusters)

In [17]:
print(len(sorted_clusters))
count = 0
for i in sorted_clusters:
    count += 1
    print("CLUSTER ", count)
    for u in i:
        #print(u,"\n",type(u[0]), type(u[1]),type(u[2]),type(u[3]),"\n")
        print(" ".join(u[2][0]))
    print()

6
CLUSTER  1
can refresh the predicted weather
know when my plots will be in danger

CLUSTER  2
can export observation data for a plot
can crud a farmer

CLUSTER  3
can filter my plots in the list

CLUSTER  4
can sort my plots in the list
can manage my irrigations and recommendations in my favorite unit
can relaunch all failed simulation

CLUSTER  5
can choose my preferred irrigation unit in my user settings
can view my irrigation recommendations in my favorite unit

CLUSTER  6
can crud plots
can edit the parameters of a plot current season



In [18]:
dfRoleFeatureSuperF["superF"] = ""

count = -1
for i in sorted_clusters:
    count += 1
    print("CLUSER ", count)
    for u in i:
        idValue = dfRoleFeatureSuperF.index[dfRoleFeatureSuperF['Feature'] == " ".join(u[2][0])].tolist()
        dfRoleFeatureSuperF.at[idValue, "superF"] = str(count)
        

CLUSER  0
CLUSER  1
CLUSER  2
CLUSER  3
CLUSER  4
CLUSER  5


In [19]:
dfRoleFeatureSuperF.sort_values(by=['superF'])


,Role,Feature,superF
0,farmer,can refresh the predicted weather,0
6,farmer,know when my plots will be in danger,0
5,farmer,can export observation data for a plot,1
10,administrateur,can crud a farmer,1
4,farmer,can filter my plots in the list,2
3,farmer,can sort my plots in the list,3
7,farmer,can manage my irrigations and recommendations in my favorite unit,3
11,administrateur,can relaunch all failed simulation,3
8,farmer,can choose my preferred irrigation unit in my user settings,4
9,farmer,can view my irrigation recommendations in my favorite unit,4


In [20]:
dfRoleFeatureSuperF

,Role,Feature,superF
0,farmer,can refresh the predicted weather,0
1,farmer,can crud plots,5
2,farmer,can edit the parameters of a plot current season,5
3,farmer,can sort my plots in the list,3
4,farmer,can filter my plots in the list,2
5,farmer,can export observation data for a plot,1
6,farmer,know when my plots will be in danger,0
7,farmer,can manage my irrigations and recommendations in my favorite unit,3
8,farmer,can choose my preferred irrigation unit in my user settings,4
9,farmer,can view my irrigation recommendations in my favorite unit,4


# FCA Multiple

In [21]:
dfRoleFeatureSuperF["Feature"]

0                                     can refresh the predicted weather
1                                                        can crud plots
2                      can edit the parameters of a plot current season
3                                         can sort my plots in the list
4                                       can filter my plots in the list
5                                can export observation data for a plot
6                                  know when my plots will be in danger
7     can manage my irrigations and recommendations in my favorite unit
8           can choose my preferred irrigation unit in my user settings
9            can view my irrigation recommendations in my favorite unit
10                                                    can crud a farmer
11                                   can relaunch all failed simulation
Name: Feature, dtype: object

## product2Feature

In [22]:
df_produits = pd.DataFrame(data={"Produit":['Vine', 'Orchard', 'Almond']})
df_produits

,Produit
0,Vine
1,Orchard
2,Almond


In [25]:
products = [
["Vine","Orchard","Almond"],
["Vine","Orchard"],
["Vine","Orchard","Almond"],
["Vine","Orchard"],
["Orchard"],
["Orchard"],
["Vine"],
["Vine","Orchard"],
["Almond"],
["Vine"],
["Vine","Almond"],
["Vine","Orchard"],
["Vine","Orchard","Almond"]
]

In [26]:
listproduits = products
listproduitsperUS = []
for elem in listproduits:
    listTmp = []
    for prod in elem:
        listTmp.append(prod)
    listproduitsperUS.append(listTmp)

df_produitsPerUS = pd.DataFrame(data={"listproduitsperUS":listproduitsperUS})

In [27]:
df_produitsPerUS

,listproduitsperUS
0,"[Vine, Orchard, Almond]"
1,"[Vine, Orchard]"
2,"[Vine, Orchard, Almond]"
3,"[Vine, Orchard]"
4,[Orchard]
5,[Orchard]
6,[Vine]
7,"[Vine, Orchard]"
8,[Almond]
9,[Vine]


In [28]:
pd.read_csv("data/ctxF_produits.csv").drop("Unnamed: 0", axis=1)["Produit"]

0       Vine
1     Almond
2    Orchard
Name: Produit, dtype: object

In [29]:
#df_product2UFeature["Produits"] = []
dfRoleFeatureSuperF["Produits"] = df_produitsPerUS

In [30]:
product2Feature = my_array = np.array(ctxRelPerCtxForms(df_produits, df_produitsPerUS))

In [31]:
print(len(product2Feature), " ", len(dfRoleFeatureSuperF["Feature"]))

13   12


In [32]:
# df_produits x df_idUS
df_product2UFeature = pd.DataFrame(my_array,
                                    columns=df_produits["Produit"],
                                    index=dfRoleFeatureSuperF["Feature"])
df_product2UFeature

ValueError: Shape of passed values is (13, 3), indices imply (12, 3)

In [ ]:
dfRoleFeatureSuperF

In [ ]:
df_product2UFeature.index

In [ ]:
def toFC(df, column):
    noDuplicate = df[column].drop_duplicates().tolist()
    df_tmp = pd.DataFrame(data={column: list(dict.fromkeys(noDuplicate))})
    df_complet = pd.DataFrame(data={column: df[column].tolist()})
    data = np.array(ctxRelPerCtxForms(df_tmp, df_complet))
    return pd.DataFrame(data, columns=noDuplicate)
    
df_superF = toFC(dfRoleFeatureSuperF, "superF").set_index(df_product2UFeature.index)
df_Role = toFC(dfRoleFeatureSuperF, "Role").set_index(df_product2UFeature.index)
dfcombined = pd.concat([df_Role, df_product2UFeature, df_superF], axis=1)#.drop(['Produits'], axis=1)
dfcombined

In [ ]:
def getFCPerProduit(df_produits, df_perRole):
    # Product x US
    dfFinal = df_perRole[df_produits["Produit"]].transpose()

    # US x Cluster
    df_usCluster = df_perRole[dict.fromkeys(dfRoleFeatureSuperF["superF"])]
        
    
    # Product x (US + Cluster)
    for newColum in dict.fromkeys(dfRoleFeatureSuperF["superF"]):
        dfFinal[newColum] = ""
            
    # For each product
    for index, row in df_produits.iterrows():
        # Produit
        #print(row["Produit"])
        #print(len(df_perRole.loc[df_perRole[row["Produit"]] == "x"]))
        if(len(df_perRole.loc[df_perRole[row["Produit"]] == "x"]) > 0):
            #print(df_perRole.loc[df_perRole[row["Produit"]] == "x"])
            
#            df_perRole.loc[df_perRole[row["Produit"]] == "x"].to_csv("CSVs/ctxF_" + "ProductUSCluster" + ".csv")
        
            # For each cluster
            for cluster in df_perRole[dict.fromkeys(dfRoleFeatureSuperF["superF"])]:
                print(cluster)
            
def getDFProductUS(df_produits, df_perRole):
    # Product x US
    return df_perRole[df_produits["Produit"]].transpose() 

getDFProductUS(df_produits, dfcombined.loc[dfcombined["farmer"] == "x"])

def getdfProductCluster(df_produits, df_perRole):
    # Product x US
    return df_perRole[df_produits["Produit"]].transpose() 


In [ ]:
for role in list(dict.fromkeys(dfRoleFeatureSuperF["Role"])):
    print("\n",role, len(dfcombined.loc[dfcombined[role] == "x"]))
    getFCPerProduit(df_produits, dfcombined.loc[dfcombined[role] == "x"])

In [ ]:

getDFProductUS(df_produits, dfcombined.loc[dfcombined["farmer"] == "x"])